# Week 8

This week, we consider Inception nets (Szegedy, et al., 2014 https://arxiv.org/abs/1409.4842), which are CNNs with so-called inception modules, which are small groupings of parallel convolutional layers that act on the same input feature map, processes it through different means, and concatenates the results. In particular, it uses convolutional layers with different-sized filters (1x1, 3x3, and 5x5 in Inception-v1), which are adept at finding patterns at different scales.

In this notebook, we implement a small version of Inception and test it on the CIFAR-10 dataset.

## MiniGoogLeNet

Note for GoogLeNet, the inception modules are not exactly sequential, so we will need to define layers a little differently as we see below, after importing some packages. And, we use "miniception" modules as per *Understanding Deep Learning Requires Re-Thinking Generalization* (Zhang, et al., 2017 http://arxiv.org/abs/1611.03530), which work well for small-dimensional image datasets.

In [1]:
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import concatenate
from tensorflow.keras.layers import add
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import add
from tensorflow.keras import backend

In [2]:
class MiniGoogLeNet:
    def convolution_module(x, K, kX, kY, stride, channelsDim, padding="same"):
        # create a CONV -> BN -> RELU sequence
        x = Conv2D(K, (kX, kY), strides = stride, padding = padding)(x)
        x = BatchNormalization(axis = channelsDim)(x)
        x = Activation('relu')(x)
        
        # return the output
        return x
    
    def inception_module(x, numberOf1x1Kernels, numberOf3x3Kernels, channelsDim):
        # define two "parallel" convolutions of size 1x1 and 3x3 concatenated across the channels dimension
        convolution_1x1 = MiniGoogLeNet.convolution_module(x, numberOf1x1Kernels, 1, 1, (1, 1), channelsDim)
        convolution_3x3 = MiniGoogLeNet.convolution_module(x, numberOf3x3Kernels, 3, 3, (1, 1), channelsDim)
        x = concatenate([convolution_1x1, convolution_3x3], axis = channelsDim)
        
        return x
        
    def downsample_module(x, K, channelsDim):
        # define a CONV and POOL and then concatenate across the channels dimension
        convolution_3x3 = MiniGoogLeNet.convolution_module(x, K, 3, 3, (2, 2), channelsDim, padding = 'valid')
        pool = MaxPooling2D((3, 3), strides = (2, 2))(x)
        x = concatenate([convolution_3x3, pool], axis = channelsDim)
        
        return x
    
    def build(width, height, depth, classes):
        inputShape = (height, width, depth)
        channelsDim = -1
        
        if backend.image_data_format() == "channels_first":
            inputShape = (depth, height, width)
            channelsDim = 1
        
        # define the model input and first CONV module
        inputs = Input(shape = inputShape)
        x = MiniGoogLeNet.convolution_module(inputs, 96, 3, 3, (1, 1), channelsDim)
        
        # two inception modules followed by a downsample module
        x = MiniGoogLeNet.inception_module(x, 32, 32, channelsDim)
        x = MiniGoogLeNet.inception_module(x, 32, 48, channelsDim)
        x = MiniGoogLeNet.downsample_module(x, 80, channelsDim)
        
        # four inception modules followed by a downsample module
        x = MiniGoogLeNet.inception_module(x, 112, 48, channelsDim)
        x = MiniGoogLeNet.inception_module(x, 96, 64, channelsDim)
        x = MiniGoogLeNet.inception_module(x, 80, 80, channelsDim)
        x = MiniGoogLeNet.inception_module(x, 48, 96, channelsDim)
        x = MiniGoogLeNet.downsample_module(x, 96, channelsDim)
        
        # two inception modules followed by global POOL and dropout
        x = MiniGoogLeNet.inception_module(x, 176, 160, channelsDim)
        x = MiniGoogLeNet.inception_module(x, 176, 160, channelsDim)
        x = AveragePooling2D((7, 7))(x)
        x = Dropout(0.5)(x)
        
        # softmax classifier
        x = Flatten()(x)
        x = Dense(classes)(x)
        x = Activation('softmax')(x)
        
        # create a model
        model = Model(inputs, x, name='MiniGoogLeNet')
        
        # return the model
        return model

### MiniGoogLeNet on CIFAR-10

Let's test it on CIFAR-10

In [3]:
# import the necessary packages
from sklearn.preprocessing import LabelBinarizer
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.datasets import cifar10
import numpy as np
import os

from sklearn.metrics import classification_report
import matplotlib.pyplot as plt

In [5]:
numberOfEpochs = 70
initialLearningRate = 0.005

def polynomial_decay(epoch):
    maxEpochs = numberOfEpochs
    baseLearningRate = initialLearningRate
    power = 1.0
    
    alpha = baseLearningRate * (1 - (epoch / float(numberOfEpochs))) ** power
    
    # return the learning rate
    return alpha
    
# load cifar10 data
((trainX, trainY), (testX, testY)) = cifar10.load_data()
trainX = trainX.astype('float')
testX = testX.astype('float')

# use mean subtraction
mean = np.mean(trainX, axis = 0)
trainX -= mean
testX -= mean

# convert labels to one-hot
lb = LabelBinarizer()
trainY = lb.fit_transform(trainY)
testY = lb.transform(testY)

callbacks = [LearningRateScheduler(polynomial_decay)]

print('[INFO] compiling model...')
opt = SGD(lr = initialLearningRate, momentum=0.9)
model = MiniGoogLeNet.build(width = 32, height = 32, depth = 3, classes = 10)
model.compile(loss = 'categorical_crossentropy', optimizer = opt, metrics=["accuracy"])

# print a model summary
print(model.summary())

# train the network
print("[INFO] training network...")
H = model.fit(trainX, trainY, validation_data = (testX, testY), batch_size = 64, epochs = numberOfEpochs,
              callbacks = callbacks, verbose = 1)

# save the network to disk
#print("[INFO] serializing network...")
#model.save('output/MiniGoogLeNet_cifar10.hdf5')

# print a classification report
print('\n Test accuracy')
predictedY = model.predict(testX)
predictedY = predictedY.argmax(axis=1)
testY = testY.argmax(axis=1)
print(classification_report(testY, predictedY, digits=4))

# plot the training loss and accuracy
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, numberOfEpochs), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, numberOfEpochs), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, numberOfEpochs), H.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, numberOfEpochs), H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend()

[INFO] compiling model...
Model: "MiniGoogLeNet"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv2d_19 (Conv2D)              (None, 32, 32, 96)   2688        input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_19 (BatchNo (None, 32, 32, 96)   384         conv2d_19[0][0]                  
__________________________________________________________________________________________________
activation_20 (Activation)      (None, 32, 32, 96)   0           batch_normalization_19[0][0]     
____________________________________________________________

This results in 85% accuracy, which is pretty good, but it seems to be overfitting.

### GoogLeNet Experiment 2: Data Augmentation

Let's see if data augmentation helps with the overfitting problem.

In [3]:
numberOfEpochs = 70
initialLearningRate = 0.005

def polynomial_decay(epoch):
    maxEpochs = numberOfEpochs
    baseLearningRate = initialLearningRate
    power = 1.0
    
    alpha = baseLearningRate * (1 - (epoch / float(numberOfEpochs))) ** power
    
    # return the learning rate
    return alpha
    
# load cifar10 data
((trainX, trainY), (testX, testY)) = cifar10.load_data()
trainX = trainX.astype('float')
testX = testX.astype('float')

# use mean subtraction
mean = np.mean(trainX, axis = 0)
trainX -= mean
testX -= mean

# convert labels to one-hot
lb = LabelBinarizer()
trainY = lb.fit_transform(trainY)
testY = lb.transform(testY)

# construct the image generator for data augmentation
aug = ImageDataGenerator(width_shift_range = 0.1, height_shift_range = 0.1,
                         horizontal_flip = True, fill_mode="nearest")

callbacks = [LearningRateScheduler(polynomial_decay)]

print('[INFO] compiling model...')
opt = SGD(lr = initialLearningRate, momentum=0.9)
model = MiniGoogLeNet.build(width = 32, height = 32, depth = 3, classes = 10)
model.compile(loss = 'categorical_crossentropy', optimizer = opt, metrics=["accuracy"])

# train the network
print("[INFO] training network...")

H = model.fit(aug.flow(trainX, trainY, batch_size=64), validation_data=(testX, testY),
              steps_per_epoch=len(trainX) // 64, epochs=numberOfEpochs, callbacks = callbacks, verbose=1)

# print a model summary
print(model.summary())

# print a classification report
print('\n Test accuracy')
predictedY = model.predict(testX)
predictedY = predictedY.argmax(axis=1)
testY = testY.argmax(axis=1)
print(classification_report(testY, predictedY, digits=4))

# plot the training loss and accuracy
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, numberOfEpochs), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, numberOfEpochs), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, numberOfEpochs), H.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, numberOfEpochs), H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend()

[INFO] compiling model...
[INFO] training network...
  ...
    to  
  ['...']
Train for 781 steps, validate on 10000 samples
Epoch 1/70
781/781 [==============================] - 36s 46ms/step - loss: 1.4885 - accuracy: 0.4570 - val_loss: 1.3241 - val_accuracy: 0.5350
Epoch 2/70
781/781 [==============================] - 46s 58ms/step - loss: 1.0807 - accuracy: 0.6133 - val_loss: 1.0269 - val_accuracy: 0.6423
Epoch 3/70
781/781 [==============================] - 47s 60ms/step - loss: 0.8980 - accuracy: 0.6833 - val_loss: 1.2337 - val_accuracy: 0.5979
Epoch 4/70
781/781 [==============================] - 287s 368ms/step - loss: 0.7859 - accuracy: 0.7247 - val_loss: 0.8290 - val_accuracy: 0.7109
Epoch 5/70
781/781 [==============================] - 38s 49ms/step - loss: 0.7043 - accuracy: 0.7562 - val_loss: 0.9168 - val_accuracy: 0.7040
Epoch 6/70
781/781 [==============================] - 32s 40ms/step - loss: 0.6400 - accuracy: 0.7788 - val_loss: 0.7356 - val_accuracy: 0.7498
Epoch 7/7

We've now cracked 90% for the first time with CIFAR-10!